In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "1"  # Set the GPUs to use

In [4]:
import torch

from transformers import (
    AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, 
)

from peft import get_peft_model, LoraConfig, TaskType, PeftModel


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /workspace/news-topic-keyphrase-generation-model-dev/.venv/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda116.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /workspace/news-topic-keyphrase-generation-model-dev/.venv/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda116.so...


/workspace/news-topic-keyphrase-generation-model-dev/.venv/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/workspace/news-topic-keyphrase-generation-model-dev/.venv/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/workspace/news-topic-keyphrase-generation-model-dev/.venv/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('Asia/Seoul')}
  warn(msg)
/workspace/news-topic-keyphrase-generation-model-dev/.venv/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:

In [5]:
NGPU = torch.cuda.device_count()
NCPU = os.cpu_count()
NGPU, NCPU

(1, 64)

In [6]:
torch.cuda.is_available()

True

In [7]:
MODEL_CHECKPOINT = 'EleutherAI/gpt-j-6b'
MODEL_CHECKPOINT = 'ainize/gpt-j-6B-float16'
MODEL_CHECKPOINT = 'ainize/kobart-news'
MODEL_CHECKPOINT = 'paust/pko-t5-base'
MODEL_CHECKPOINT = 'EleutherAI/polyglot-ko-1.3b'

In [8]:
config = AutoConfig.from_pretrained(MODEL_CHECKPOINT)

In [9]:
config.architectures[0].lower()

'gptneoxforcausallm'

In [11]:
# model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT, config=config)
model = AutoModelForCausalLM.from_pretrained(MODEL_CHECKPOINT, config=config)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

In [8]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

In [9]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 884736 || all params: 276463872 || trainable%: 0.3200186677556191


In [11]:
model.save_pretrained('./.temp/peft_save_test')

In [20]:
tokenizer.encode('<pad>')

[0, 3, 1]

In [11]:
tokenizer.decode([0])

'<pad>'

In [28]:
test_case = '정부세종청사 교육부 전경./사진=뉴스1[파이낸셜뉴스] 교육부가 학부모의 체육 관련 사교육 수요를 공교육으로 흡수시키기 위한 학교체육 활성화 방안을 내놓았다. 학교에서 제공하는 체육 프로그램을 다양화해 학생들 신체활동을 유도한다는 방침이다.교육부는 10일 2023년 학교체육 활성화 추진 계획을 수립, 시도교육청에 배포했다.지난해 초중고 사교육비 조사결과에 따르면 학생 1인당 예체능·취미교양 월평균 사교육비는 9만8000원으로 전년 대비 17.8% 증가했다. 교육부는 다양한 체육활동 프로그램을 활성화해 최근 늘어난 사교육 수요를 학교 안에서 충족시킨다는 구상을 세우고 있다.이를 위해 교육부는 특별교부금 예산 528억원을 지원, 학교스포츠클럽과 전국 학교스포츠클럽 축전의 종목 수를 확대하기로 했다. 지난해 기준 1개교당 평균 11개, 전국 12만8000개 수준인 학교스포츠클럽을 올해 1개교당 평균 20개, 전국 23만6000개로 늘리겠다는 것이다. 또한 체육온활동을 도입해 방과후 체육활동 확대를 추진한다.교육부는 올해부터 콘 축구, 농구 패스 게임 등 148종의 신체활동 프로그램을 활용한 \'체육온동아리\'를 도입해 평소 체육활동에 소극적인 학생들의 참여를 유도한다.아울러 체육활동 앱에 서킷트레이닝, 킨볼 등 288종의 콘텐츠를 제공해 학생들이 신체활동을 수행할 수 있도록 돕는다.현재 초등학교 5학년부터 실시하는 건강체력평가(PAPS)는 초등학교 1~4학년 학생으로 대상을 확대하기 위해 평가기준을 개발할 예정이다. 건강체력교실 등 체력향상프로그램도 학생 맞춤형으로 상시 운영한다.학생선수 학습권 보장을 위해선 50명 규모의 학습지원멘토단을 시범운영한다. 지난해 34명 수준이었던 진로상담멘토풀도 50명으로 확대한다.이외에도 온·오프라인 상담을 강화해 초등 학생선수용 \'이스쿨(e-school)\' 프로그램을 개발·제공한다.6개 체고를 대상으로는 고교학점제 준비학교를 운영해 체육진로 공동교육과정 운영 모델 방안을 마련할 방침이다.이주호 부총리 겸 교육부장관은 "늘어나는 체육활동 사교육 수요를 공교육 내에서 흡수할 수 있도록 관계부처·시도교육청 및 체육 유관기관·단체와 적극적으로 협력하여 모든 학생과 학부모들이 만족할 수 있도록 학교 체육활동 활성화에 힘쓰겠다"고 전했다.'

In [29]:
len(tokenizer.encode(test_case, truncation=True))
len(tokenizer(test_case, truncation=True)['input_ids'])

741